In [14]:
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
from IPython.display import Audio  # We only need the Audio 'Class' from this package
import librosa

import os 
import math
import json

In [15]:
DATASET_PATH = "genres"
JSON_PATH = "data.json"
SAMPLE_RATE = 22050
DURATION = 30 # measure in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

def save_mfcc(

    dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_length=512, num_segments=5

):

    data = {"mapping": [], "mfcc": [], "labels": []}
    
    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length) # ceil 1.2-> 2
    
    # loop through all the genres

    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

        # ensure we are not at the root level
        if dirpath is not dataset_path:
            pass

            # save the semantic label
            dirpath_components = dirpath.split("/")  # genre/blues => ["genre", "blues"]
            semantic_label = dirpath_components[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing {}".format(semantic_label))

            # process files for a specific genre
            for f in filenames:

                file_path = os.path.join(dirpath, f)
                signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)

                # for backup
                # read = sf.read(file_path)
                # signal, sr = read[0], SAMPLE_RATE

                for s in range(num_segments):
                    start_sample = num_samples_per_segment * s
                    finish_sample = start_sample + num_samples_per_segment


                    mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample], sample_rate = 22050, n_mfcc=13, n_fft=2048, hop_length=512)                  
                    mfcc = mfcc.T 
                    
                    #store mfcc for segment if it has the expected length
                    if len(mfcc) == expected_num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i-1)
                        print("{}, segment:{}".format(file_path,s+1))

    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)
        
                

In [16]:
save_mfcc(DATASET_PATH,JSON_PATH,num_segments=10)


Processing genres\blues


TypeError: mfcc() takes 0 positional arguments but 1 positional argument (and 1 keyword-only argument) were given